OrderBook Demo
-------------

Examples of placing orders, fill probability estimation, and simple strategy backtests

In [1]:
#Mosie Schrem
import pandas as pd
import numpy as np
import os
import math
import sys
from matplotlib import pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
sys.path.insert(0, os.path.join(os.getcwd(), 'code'))

import OrderBook as ob
import OrderUtil as ou
import FillProbabilitySimulator as fps
import TradingStrategySimulator as tss

Examples of creating and processing orders:
--------

In [2]:
#load in intc OrderBook data
book = ob.OrderBook(message_filename='INTC_message.csv', orderbook_filename='INTC_OrderBook.csv')

In [3]:
limit_book = book.limit_order_book()
limit_book['mid'] = limit_book['ap1'] - limit_book['bp1']
limit_book = limit_book.loc[(limit_book['ap1'] - limit_book['bp1']).shift(1) >= 
                            limit_book['ap1'] - limit_book['bp1']]
limit_book = limit_book.loc[limit_book['mid'].shift(1) != limit_book['mid']]
limit_book['label'] = [1 if limit_book['mid'].shift(1).values[i] < limit_book['mid'].values[i] else -1
                      for i in range(len(limit_book))]

In [4]:
print(len(limit_book))

2956


Create an ask order at level 2, at time 38000
-------
We reevaluate order status at every BookUpdate 

In [5]:
order = ou.BookUpdatesOrder(orderbook=book, numupdates=1, timestamp=38000, level=2, is_buy=False)
print(order.get_opening_stats())

{'price': 273800, 'start_time': 38000, 'is_buy': False, 'queue_position': 16646, 'orderstate': 'open', 'level': 2, 'start_index': 131939}


In [6]:
#process 100 updates to orderbook
for i in range(100):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38014.459199206, 'is_buy': False, 'queue_position': 16446, 'orderstate': 'open', 'level': 2, 'current_index': 132039}


In [7]:
#process another 300 updates
for i in range(300):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38015.006473335, 'is_buy': False, 'queue_position': 16346, 'orderstate': 'open', 'level': 2, 'current_index': 132339}


In [8]:
#process until executed or cancelled
while(order.order_type('open')):
    order.update()
    
#print stats at time when order was executed
print(order.get_closing_stats())

{'price': 273800, 'time': 38167.621597535, 'is_buy': False, 'queue_position': 0, 'orderstate': 'executed', 'level': 1, 'end_index': 134180}


We can also create orders by timestep of fixed size
--------

Create an ask order at level 2, at time 38000, reevaluating order status every 0.5 seconds

In [9]:
order = ou.TimeOrder(orderbook=book, delta_t=0.5, timestamp=38000, level=2, is_buy=False)
print(order.get_opening_stats())

{'price': 273800, 'start_time': 38000, 'is_buy': False, 'queue_position': 16646, 'orderstate': 'open', 'level': 2, 'start_index': 131939}


In [10]:
#process 50 seconds of data
for i in range(100):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38049.987429576, 'is_buy': False, 'queue_position': 13946, 'orderstate': 'open', 'level': 3, 'current_index': 132820}


In [11]:
#process until executed or cancelled
while(order.order_type('open')):
    order.update()
#print stats at time when order was executed
print(order.get_closing_stats())

{'price': 273800, 'time': 38167.621597535, 'is_buy': False, 'queue_position': 0, 'orderstate': 'executed', 'level': 1, 'end_index': 134180}


Fill Probability Simulation Example:
------

In the following example, we quote buy and sell orders in the inside market, and reevaluate every 100 BookUpdates

In [12]:
probability_simulator = fps.FillProbabilitySimulator(numupdates=100, order_tuple=([1] , [1]),
                                                 orderbook=book, t_start=34200.01, t_end=50000)
probability_simulator.generate_matrices(num_samples=10000)

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples processed so far: 7000.0
samples processed so far: 8000.0
samples processed so far: 9000.0
samples processed so far: 10000.0


Display empirical fill probabilities for given midprice movement -1, 0, and 1

Note that bid_level_i_ask_level_j refers to bid orders filled up to level i and ask orders filled up to level j
bid_level_0 (ask_level_0) refers to no bids (asks) filled.

In [13]:
fill_matrix = probability_simulator.get_prob_matrix()
fill_matrix

y,-1,0,1
orders_executed,,,
bid_level_0_ask_level_0,0.391122,0.933693,0.375638
bid_level_0_ask_level_1,0.000000,0.032403,0.618062
bid_level_1_ask_level_0,0.596581,0.032103,0.002100
bid_level_1_ask_level_1,0.012298,0.001800,0.004200


Display frequencies of midprice movements in our sample

In [14]:
print(probability_simulator.get_quantities())

{-1: 3334, 0: 3333, 1: 3333}


Here we display the cumulative distribution of our fill probability matrix

In [15]:
cum_mat = probability_simulator.get_cum_prob_matrix()
cum_mat

y,-1,0,1
orders_executed,,,
bid_level_0_ask_level_0,1.000000,1.000000,1.000000
bid_level_0_ask_level_1,0.012298,0.034203,0.622262
bid_level_1_ask_level_0,0.608878,0.033903,0.006301
bid_level_1_ask_level_1,0.012298,0.001800,0.004200


Display PNLs

In [16]:
pnls = probability_simulator.get_pnls()
pnls

y,-1,0,1
orders placed,,,
"bids: [1], asks: [1]",-20.0,0.06,-20.44


Loss Function Estimation
-------------

Here we approximate our loss function to feed into our Neural Network using a similar approach to the fill probability simulation.

We use the same strategy as we did for the above backtest example

In [17]:
strategy = {1: ([1], [2]), 0: ([1], [1]), -1 : ([2], [1])}
loss_func = fps.LossFunction(strategy=strategy, book=book, numupdates=100, t_end=50000, uniform_sampling=True)
loss_func.generate_loss_function_and_fill_probabilities(num_samples=10000)

simulating strategy for case yhat = 1...

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples processed so far: 7000.0
samples processed so far: 8000.0
samples processed so far: 9000.0
samples processed so far: 10000.0
simulating strategy for case yhat = 0...

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples processed so far: 7000.0
samples processed so far: 8000.0
samples processed so far: 9000.0
samples processed so far: 10000.0
simulating strategy for case yhat = -1...

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples proce

Loss Matrix Below based on average PNL in given case (-loss are cases where our strategy yields a profit)

In [18]:
loss_function_matrix = loss_func.get_loss_matrix()
loss_function_matrix.head()

y_true:,-1,0,1
y_predicted,,,
-1,0.2,-0,20.98
0,19.8,-0.09,20.81
1,20.78,-0.03,0.27


Fill Probabilities for each strategy

In [19]:
for key in strategy:
    print("\nFill probabilities for yhat: " + str(key) + ", and trade: " + str(strategy[key]))
    print(loss_func.get_fill_probabilities()[key])


Fill probabilities for yhat: 1, and trade: ([1], [2])
y                              -1         0         1
orders_executed                                      
bid_level_0_ask_level_0  0.393821  0.970297  0.972097
bid_level_0_ask_level_2  0.000000  0.000900  0.021302
bid_level_1_ask_level_0  0.606179  0.028803  0.004200
bid_level_1_ask_level_2  0.000000  0.000000  0.002400

Fill probabilities for yhat: 0, and trade: ([1], [1])
y                              -1         0         1
orders_executed                                      
bid_level_0_ask_level_0  0.397121  0.937894  0.365137
bid_level_0_ask_level_1  0.000000  0.032403  0.626463
bid_level_1_ask_level_0  0.592681  0.027003  0.002400
bid_level_1_ask_level_1  0.010198  0.002700  0.006001

Fill probabilities for yhat: -1, and trade: ([2], [1])
y                              -1         0         1
orders_executed                                      
bid_level_0_ask_level_0  0.965807  0.966397  0.371737
bid_level_0_ask_level_1 